# Shell scripts for running each tool


## Ground truth dataset
#### 6.4mil SNPs, 2504 individuals

Data downloaded from: 


VCF file: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz


Population/sample metadata: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel

In [ ]:
# Getting text file with only sample IDs and what super population it belongs to
awk '{print $1, $3}' all_samples.txt > all_sample_pop.info

## Benchmarking time run time, output, standard error, and memory usage

I may do multiple datasets with varying SNP amounts to see how they compare. I will use three datasets with varying SNP amounts to see how run-time and memory usage is affected. I downloaded each VCF file from these locations using wget. For the ground truth, I had to use two different datasets from the same populations and source but different chromosomes due to large file sizes.

#### Datasets used:
##### ~49,000SNPs
e.vcf: https://www.frontiersin.org/journals/genetics/articles/10.3389/fgene.2023.1092066/full#supplementary-material;
##### ~1.1mil SNPs
https://www.internationalgenome.org/data-portal/search?q=chr22
##### ~2.3mil SNPs
https://www.internationalgenome.org/data-portal/data-collection/ggvp-grch37

#### Ground truth
ADMIXTURE: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr21.phase3_integrated_v1b.20130502.genotypes.vcf.gz


VCF2PCACluster and PLINK2: ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr1.phase3_integrated_v1b.20130502.genotypes.vcf.gz


### VCF2PCACluster scripts
#### Ground truth

In [ ]:
# puts job in the background and also outputs the log to a specific folder to track progress and how it ran
nohup ~/benchmarking_project/VCF2PCACluster/bin/VCF2PCACluster -InVCF ground_truth.vcf.gz -OutPut ~/benchmarking_project/final_project/VCF2PCACluster_results/ground_truth_all_bestK -InSampleGroup all_sample_pop.info -BestKManually  5 > ~/benchmarking_project/final_project/ground_truth/vcf2pcacluster_truth_bestK.log 2>&1 &

#### Real data

In [ ]:
~/benchmarking_project/VCF2PCACluster/bin/VCF2PCACluster -InVCF real_50000.vcf -OutPut ~/benchmarking_project/final_project/VCF2PCACluster_results/vcf2pcacluster_real_50000

~/benchmarking_project/VCF2PCACluster/bin/VCF2PCACluster -InVCF real_3.vcf.gz -OutPut ~/benchmarking_project/final_project/VCF2PCACluster_results/vcf2pcacluster_real_3

# large file, so ran it in the background
nohup ~/benchmarking_project/VCF2PCACluster/bin/VCF2PCACluster -InVCF real_truth_4pop.genotypes.vcf.gz -OutPut ~/benchmarking_project/final_project/VCF2PCACluster_results/vcf2pcacluster_real_truth_4pop > ~/benchmarking_project/final_project/logs/vcf2pcacluster_real_truth_4pop.log 2>&1 &



### ADMIXTURE Scripts

#### Ground Truth

##### Cross validation script for which K value is best
I am only testing 3-6 for K ancestral populations because I know my data has a population of 5, but it would be interesting to see if it detects any populations above 5. 

In [ ]:
# Getting ped/map files 
nohup plink --vcf ground_truth.vcf.gz --recode --out ground_truth &
nohup plink --file ground_truth --make-bed --out ground_truth &

# converting to .bed, issues with using .ped/.map files with admixture
plink2 --vcf ground_truth2.vcf.gz --max-alleles 2 --make-bed --out ground_truth2

Had to run each K-fold cross validation in parallel and loosen the C regularization parameter due to run-time issues.

In [ ]:
#!/bin/bash

# Number of processors to use (half of total, adjust based on your system specs)
NUM_THREADS=12
CONV_CRIT=0.1

# Run ADMIXTURE in parallel for K values
for K in 3 4 5 6; do
    nohup ~/benchmarking_project/admixture/admixture/admixture \
        --cv ground_truth2.bed $K -j$NUM_THREADS -C $CONV_CRIT \
        > ground_truth_log${K}.out 2>&1 &
done

echo "ADMIXTURE jobs submitted in parallel for K=3 to 6 with $NUM_THREADS threads each."

In [ ]:
grep -h CV ground_truth_log*.out # to view CV error to choose which K is best for further analysis
'''
CV error (K=3): 0.06388
CV error (K=4): 0.06241
CV error (K=5): 0.06188
CV error (K=6): 0.06302
''' # Best K = 5

#### Real data

In [ ]:
# converting vcf to ped file format then to .bed
plink --vcf real_50000.vcf --recode --out real_50000 --chr-set 50
plink --file new_real_50000 --make-bed --out real_50000 --chr-set 50

# removing snps with high missingness
plink --bfile real_50000 --geno 0.05 --make-bed --out filtered_real_50000 --chr-set 50

for K in 1 2 3 4 5 6 7; \
    do ./admixture --cv ~/benchmarking_project/final_project/filtered_real_50000.bed $K | tee real_50000log${K}.out; done
grep -h CV real_50000log*.out
'''Writing output files.
CV error (K=1): 0.60157
CV error (K=2): 0.59049
CV error (K=3): 0.57551
CV error (K=4): 0.59876
CV error (K=5): 0.60458
CV error (K=6): 0.62582
CV error (K=7): 0.61644''' # K = 3 is best

In [ ]:
# converting vcf to ped file format then to .bed
plink --vcf real_3.vcf.gz --recode --out real_3 
plink --file real_3 --make-bed --out real_3
# removing snps with high missingness
plink --bfile real_3 --geno 0.05 --make-bed --out filtered_real_3


nohup bash -c 'for K in 1 2 3 4 5 6; \
do ~/benchmarking_project/admixture/admixture/admixture --cv ~/benchmarking_project/final_project/filtered_real_3.ped $K | tee real_3log${K}.out; done' &
grep -h CV real_3log*.out # to view CV error to choose which K is best 

In [ ]:
# converting vcf to ped file format then to .bed
plink --vcf real_truth_4pop.genotypes.vcf.gz --recode --out real_truth_4pop
plink --file real_truth_4pop --make-bed --out real_truth_4pop

nohup bash -c 'for K in 1 2 3 4 5; \
do ~/benchmarking_project/admixture/admixture/admixture --cv ~/benchmarking_project/final_project/real_truth_4pop.bed $K | tee real_truth_log${K}.out; done' &
grep -h CV real_truth_log*.out # to view CV error to choose which K is best 

### PLINK2 Scripts

#### Ground Truth

In [ ]:
nohup plink2 --ped ground_truth.ped --map ground_truth.map --pca 10 --out ~/benchmarking_project/final_project/plink2_results/ground_truth_all &

#### Real Data for Benchmarking

In [ ]:
plink2 --ped real_50000.ped --map real_50000.map --pca 10 --out ~/benchmarking_project/final_project/plink2_results/real_50000_output

In [ ]:
plink2 --ped real_3.ped --map real_3.map --pca 10 --out ~/benchmarking_project/final_project/plink2_results/real_3_output

In [ ]:
plink2 --ped real_truth_4pop.ped --map real_truth_4pop.map --pca 6 approx --out ~/benchmarking_project/final_project/plink2_results/real_truth_4pop_output